In [ ]:
import cv2
import pytesseract
import pandas as pd
import os
from datetime import datetime
from ultralytics import YOLO
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model
import numpy as np
import tensorflow as tf

# Paths
INPUT_FOLDER = '/kaggle/input/car-number-plate-images/'  # Adjust for your dataset
OUTPUT_CSV = '/kaggle/working/plate_results.csv'

# Setup pytesseract
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# Load YOLOv5
yolo_model = YOLO("yolov5s.pt")  # Replace with your license plate-trained model

# Load InceptionResNetV2 and VGG16
inception_model = tf.keras.applications.InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg')
vgg_model = VGG16(weights='imagenet', include_top=False, pooling='avg')

# Function for OCR

def perform_ocr(plate_img):
    gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    text = pytesseract.image_to_string(thresh, config='--psm 7')
    return ''.join(filter(str.isalnum, text))

# Function to get features from CNN

def extract_features(model, image):
    resized = cv2.resize(image, (224, 224))
    img_array = img_to_array(resized)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    return features.flatten()

# Process each image
results = []

for filename in os.listdir(INPUT_FOLDER):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(INPUT_FOLDER, filename)
        image = cv2.imread(img_path)

        # --- 1. YOLOv5 Detection ---
        yolo_detections = yolo_model.predict(source=image, conf=0.5, save=False)

        for det in yolo_detections:
            for box in det.boxes.data:
                x1, y1, x2, y2, score, class_id = box.tolist()
                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                plate_img = image[y1:y2, x1:x2]
                plate_text = perform_ocr(plate_img)

                # --- 2. InceptionResNetV2 Features ---
                inc_features = extract_features(inception_model, plate_img)

                # --- 3. VGG16 Features ---
                vgg_features = extract_features(vgg_model, plate_img)

                results.append({
                    "filename": filename,
                    "plate_text": plate_text,
                    "timestamp": datetime.now().isoformat(),
                    "bbox": f"{x1},{y1},{x2},{y2}",
                    "model": "YOLOv5 + OCR",
                    "inception_feature_0": inc_features[0],  # Just storing first few features for sample
                    "vgg_feature_0": vgg_features[0]
                })

# Save results
results_df = pd.DataFrame(results)
results_df.to_csv(OUTPUT_CSV, index=False)
print("Detection and OCR complete. Results saved to:", OUTPUT_CSV)
results_df.head()
